# Discovering the Parent Hamiltonian of a Perfect MPS via Optimization

## Overview
This notebook simulates a sophisticated scenario where we have a quantum state that is guaranteed to be a perfect Matrix Product State (MPS), and our goal is to discover its parent Hamiltonian using a variational, optimization-based approach.

The process is as follows:
1.  **Generate a Perfect MPS State:** We create `|psi_initial>` by directly contracting a chain of random tensors. This ensures our "unknown" state has an exact MPS representation with a known `BOND_DIMENSION`.
2.  **Define Parameterized Projectors:** We define a parameterized unitary `U(theta)` and a static base projector $P_{base}$ of a specified size and rank.
3.  **Optimize Projectors via VQA:** For each local group of qubits, we run a simulated Variational Quantum Algorithm (VQA). A classical optimizer (COBYLA) seeks parameters `theta` to minimize the expectation value $\langle\psi_{initial}|P(theta)|\psi_{initial}\rangle$, which is calculated via simulated measurements.
4.  **Construct Parent Hamiltonian:** The Hamiltonian $H$ is built by summing the successfully optimized projectors $P^*$.
5.  **Verification:** We find the ground state of our discovered $H$ and verify that it has high fidelity with the original `|psi_initial>`, confirming our method worked.

## 1. Imports and Setup

In [22]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp, state_fidelity, random_unitary
import itertools 

try:
    from scipy.optimize import minimize
except ModuleNotFoundError:
    print("ERROR: scipy is not installed. Please install it: pip install scipy")
    def minimize(*args, **kwargs):
        raise RuntimeError("scipy.optimize.minimize not found. Please install scipy.")

## 2. Configuration Parameters

In [29]:
# --- Main Experiment Configuration ---
NUM_QUBITS = 8
BOND_DIMENSION = 4        # Bond dimension of the initial MPS state
PROJECTOR_QUBIT_COUNT = 5  # Number of qubits each projector acts on

# Set projector rank based on projector size
# Rank 2**(N-1) is a projector onto the even parity subspace for the Z...Z operator
PROJECTOR_RANK = 2**(PROJECTOR_QUBIT_COUNT - 1)

# --- Optimizer and Ansatz Configuration ---
# Set unitary complexity based on projector size
PROJECTOR_U_REPS = PROJECTOR_QUBIT_COUNT
SCIPY_COBYLA_MAXITER = 40000
SCIPY_COBYLA_TOL = 1e-4
SEED = 42

np.random.seed(SEED)
print(f"--- Discovering Parent Hamiltonian for a Perfect MPS ---")
print(f"Config: Qubits={NUM_QUBITS}, Bond Dim={BOND_DIMENSION}")
print(f"Projectors: Size={PROJECTOR_QUBIT_COUNT}, Rank={PROJECTOR_RANK}, U_reps={PROJECTOR_U_REPS}\n")

--- Discovering Parent Hamiltonian for a Perfect MPS ---
Config: Qubits=8, Bond Dim=4
Projectors: Size=5, Rank=16, U_reps=5



## 3. Step 1: Generate a Perfect MPS State `|psi_initial>`
We generate the statevector by directly contracting a series of random tensors. This guarantees it is a perfect MPS with the specified `BOND_DIMENSION`.

In [30]:
def generate_perfect_mps_statevector(num_qubits, bond_dim, physical_dim=2):
    """Generates a statevector from a random MPS tensor train."""
    mps_tensors = []
    # Left-most tensor (vector)
    rand_mat_first = np.random.randn(physical_dim, bond_dim) + 1j * np.random.randn(physical_dim, bond_dim)
    q_first, _ = np.linalg.qr(rand_mat_first.T)
    mps_tensors.append(q_first.T.reshape(physical_dim, 1, bond_dim))

    # Bulk tensors
    for _ in range(num_qubits - 2):
        rand_mat = np.random.randn(physical_dim * bond_dim, bond_dim) + 1j * np.random.randn(physical_dim * bond_dim, bond_dim)
        q_mid, _ = np.linalg.qr(rand_mat)
        mps_tensors.append(q_mid.reshape(physical_dim, bond_dim, bond_dim))

    # Right-most tensor (vector)
    rand_vec_last = np.random.randn(physical_dim * bond_dim) + 1j * np.random.randn(physical_dim * bond_dim)
    mps_tensors.append((rand_vec_last / np.linalg.norm(rand_vec_last)).reshape(physical_dim, bond_dim, 1))
    
    # Contract the MPS tensors to get the full statevector
    current_vector = mps_tensors[0][:, 0, :]
    for i in range(1, num_qubits):
        tensor = mps_tensors[i]
        current_vector = np.tensordot(current_vector, tensor, axes=([1], [1]))
        current_vector = current_vector.reshape(-1, tensor.shape[2])
        current_vector = current_vector / np.linalg.norm(current_vector)
    final_state_vector = current_vector.flatten()
    return Statevector(final_state_vector)

print("Step 1: Generating a perfect MPS state...")
psi_initial_vector = generate_perfect_mps_statevector(NUM_QUBITS, BOND_DIMENSION)
print(f"  |psi_initial> statevector created. Norm: {np.linalg.norm(psi_initial_vector.data):.4f}")

Step 1: Generating a perfect MPS state...
  |psi_initial> statevector created. Norm: 1.0000


## 4. Step 2: Variational Optimization of Projectors $P^*$

We employ Variational Quantum Algorithms (VQA) to optimize unitary transformations $U$ that minimize the expectation value $\langle \psi_{\text{initial}} | P^* | \psi_{\text{initial}} \rangle$, where:
- $P^* = U P_0 U^\dagger$ is the optimized projector acting on $k$-qubit subsystems
- $P_0$ is a base projector of rank $2^{k-1}$ (half the Hilbert space dimension)
- Minimization ensures $|\psi_{\text{initial}}\rangle$ resides in the kernel of $P^*$

This constructs the core component $H = \sum_i P_i^*$ for the parent Hamiltonian. The optimization uses:
- COBYLA classical optimizer with tolerance $1\times10^{-4}$
- TwoLocal ansatz with $k-1$ layers of RyRzCX gates
- Random parameter initialization with seed management


In [31]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp, state_fidelity
import itertools 
from scipy.optimize import minimize

def run_parent_hamiltonian_test(
    num_qubits, 
    psi_initial_vector, 
    proj_qubit_count, 
    # proj_rank is no longer needed as an argument, as it's fixed by the projector type
    proj_u_reps, 
    run_seed,
    optimizer_tol,
    optimizer_maxiter
):
    """
    Runs one instance of the parent Hamiltonian test using a robust ZZ...Z-parity
    projector and returns the final fidelity.
    """
    # The rank is fixed by the projector type: 2**(proj_qubit_count - 1)
    proj_rank = 2**(proj_qubit_count - 1)
    
    print(f"\n--- Starting New Test Run (Seed: {run_seed}) ---")
    print(f"Config: N_qubits={num_qubits}, Proj_Size={proj_qubit_count}, Proj_Rank={proj_rank}, U_Reps={proj_u_reps}")
    np.random.seed(run_seed)
    
    # --- 2. Define and Optimize Projectors ---
    print("\n[Step 2: Defining and Optimizing Projectors]")
    qubit_groups = [tuple((i + j) % num_qubits for j in range(proj_qubit_count)) for i in range(num_qubits)]
    print(f"  Defined qubit groups: {qubit_groups}")
    
    u_ansatz = TwoLocal(proj_qubit_count, ['ry', 'rz'], 'cx', 'linear', reps=proj_u_reps)
    num_u_params = u_ansatz.num_parameters
    print(f"  Unitary ansatz U(theta) has {num_u_params} parameters.")
    
    optimized_projectors = []
    for q_group in qubit_groups:
        q_group_str = "".join(map(str, q_group))
        print(f"\n  Optimizing projector for group {q_group}...")

        # --- MODIFIED BASE PROJECTOR CONSTRUCTION: 0.5 * (I + Z...Z) ---
        # Identity string for all qubits
        s_identity_full = ['I'] * num_qubits
        
        # String with Z on all qubits in the group
        s_z_on_group_full = ['I'] * num_qubits
        for q_idx in q_group:
            s_z_on_group_full[q_idx] = 'Z'
        
        # Construct the Pauli list for the SparsePauliOp
        # We do not reverse the string [::-1] here, assuming direct index mapping.
        # This creates a little-endian string if read from right to left.
        pauli_list_for_proj = [
            ("".join(s_identity_full)[::-1], 0.5),
            ("".join(s_z_on_group_full)[::-1], 0.5)
        ]
        base_projector_spo = SparsePauliOp.from_list(pauli_list_for_proj)
        # --- END OF MODIFICATION ---

        print(f"    Base projector for group {q_group} created (Rank {proj_rank}, Type: ZZ...Z Parity).")
        # print(f"    DEBUG: Base projector Pauli terms: {base_projector_spo.paulis}") # Now this will just be ['II...I', 'ZZ...I...']

        # --- Cost Function Definition ---
        def cost_func(params):
            try:
                u_circ = u_ansatz.assign_parameters(params)
                u_full_qc = QuantumCircuit(num_qubits)
                u_full_qc.compose(u_circ, qubits=list(q_group), inplace=True)
                u_op = Operator(u_full_qc)
                p_op = u_op @ base_projector_spo @ u_op.adjoint()
                expectation_value = psi_initial_vector.expectation_value(p_op)
                print(f"    Cost function evaluated: {expectation_value:.6e}", end="\r")
                return expectation_value
            except Exception as e:
                print(f"    ERROR in cost function: {e}")
                return 1e6

        # --- Optimization ---
        init_params = np.random.rand(num_u_params) * 2 * np.pi
        print(f"    Starting COBYLA optimization...")
        try:
            opt_res = minimize(
                fun=cost_func, 
                x0=init_params, 
                method='COBYLA',
                tol=optimizer_tol, 
                options={'maxiter': optimizer_maxiter, 'disp': False}
            )
            print(f"    Optimization finished. Success: {opt_res.success}, Evals: {opt_res.nfev}, Final cost: {opt_res.fun:.6e}")
            
            if opt_res.success or (not np.isnan(opt_res.fun) and opt_res.fun < 1.0):
                u_circ_opt = u_ansatz.assign_parameters(opt_res.x)
                u_full_opt = QuantumCircuit(num_qubits)
                u_full_opt.compose(u_circ_opt, qubits=list(q_group), inplace=True)
                u_op_opt = Operator(u_full_opt)
                p_star_op = u_op_opt @ base_projector_spo @ u_op_opt.adjoint()
                optimized_projectors.append(p_star_op)
            else:
                print(f"    WARNING: Optimization for P_{q_group_str} did not meet success criteria.")

        except Exception as e_opt:
            print(f"    ERROR: An exception occurred during optimization: {e_opt}")
            pass

    # --- 3. Find Ground State of H = sum(P*) ---
    print("\n[Step 3: Finding Ground State of Hamiltonian]")
    if not optimized_projectors:
        print("  No projectors were successfully optimized. Cannot continue.")
        return np.nan
    
    print(f"  Summing {len(optimized_projectors)} optimized projectors to form Hamiltonian H...")
    H_op = Operator(np.zeros((2**num_qubits, 2**num_qubits)))
    for proj in optimized_projectors:
        H_op += proj

    print("  Diagonalizing H matrix...")
    eigvals, eigvecs = np.linalg.eigh(H_op.data)
    gs_energy = eigvals[0]
    gs_gap = eigvals[1] - gs_energy if len(eigvals) > 1 else float('inf')
    gs_vector = Statevector(eigvecs[:, 0])
    
    print(f"  Ground state energy found: {gs_energy:.6e}")
    print(f"  Energy gap to first excited state: {gs_gap:.6f}")
    if gs_gap < 1e-9:
        print("  WARNING: Ground state may be degenerate.")
    else:
        print("  Ground state appears to be unique.")

    # --- 4. Final Fidelity Calculation ---
    print("\n[Step 4: Final Fidelity Calculation]")
    final_fidelity = state_fidelity(psi_initial_vector, gs_vector)
    print(f"  Fidelity(|psi_initial>, |found_GS>): {final_fidelity:.8f}")
    
    return final_fidelity


fidelity = run_parent_hamiltonian_test(
    NUM_QUBITS, 
    psi_initial_vector, 
    PROJECTOR_QUBIT_COUNT,
    PROJECTOR_U_REPS, # Note: proj_rank is now calculated inside
    SEED,
    SCIPY_COBYLA_TOL,
    SCIPY_COBYLA_MAXITER )


--- Starting New Test Run (Seed: 42) ---
Config: N_qubits=8, Proj_Size=5, Proj_Rank=16, U_Reps=5

[Step 2: Defining and Optimizing Projectors]
  Defined qubit groups: [(0, 1, 2, 3, 4), (1, 2, 3, 4, 5), (2, 3, 4, 5, 6), (3, 4, 5, 6, 7), (4, 5, 6, 7, 0), (5, 6, 7, 0, 1), (6, 7, 0, 1, 2), (7, 0, 1, 2, 3)]
  Unitary ansatz U(theta) has 60 parameters.

  Optimizing projector for group (0, 1, 2, 3, 4)...
    Base projector for group (0, 1, 2, 3, 4) created (Rank 16, Type: ZZ...Z Parity).
    Starting COBYLA optimization...


KeyboardInterrupt: 

In [ ]:
# --- Optimizer Configuration ---
# L-BFGS-B is a gradient-based method
OPTIMIZER_METHOD = 'L-BFGS-B'
OPTIMIZER_MAXITER = 80  # Gradient-based methods often need fewer iterations
OPTIMIZER_TOL = 1e-6     # Can often achieve higher precision

In [ ]:
# You can keep this at the top level of your script/notebook
def cost_func_for_grad(params, u_ansatz, base_projector_spo, num_qubits, q_group, psi_initial_vector):
    """A top-level, pickle-able cost function for evaluating a single point."""
    try:
        u_circ = u_ansatz.assign_parameters(params)
        u_full_qc = QuantumCircuit(num_qubits)
        u_full_qc.compose(u_circ, qubits=list(q_group), inplace=True)
        u_op = Operator(u_full_qc)
        p_op = u_op @ base_projector_spo @ u_op.adjoint()
        expectation_value = np.real(psi_initial_vector.expectation_value(p_op))
        print(f"    Cost function evaluated: {expectation_value:.6e}", end="\r")
        return expectation_value
    except Exception:
        return 1e6

# Main function, now with parameter-shift logic
def run_parent_hamiltonian_test(
    num_qubits, 
    psi_initial_vector, 
    proj_qubit_count, 
    proj_u_reps, 
    run_seed,
    optimizer_tol,
    optimizer_maxiter
):
    """
    Runs the parent Hamiltonian test using a gradient-based optimizer
    with the parameter-shift rule.
    """
    proj_rank = 2**(proj_qubit_count - 1)
    print(f"\n--- Starting New Test Run (Seed: {run_seed}) ---")
    print(f"Config: N_qubits={num_qubits}, Proj_Size={proj_qubit_count}, U_Reps={proj_u_reps}")
    np.random.seed(run_seed)
    
    # --- Define and Optimize Projectors ---
    print("\n[Step 2: Defining and Optimizing Projectors with Parameter-Shift Rule]")
    qubit_groups = [tuple((i + j) % num_qubits for j in range(proj_qubit_count)) for i in range(num_qubits)]
    u_ansatz = TwoLocal(proj_qubit_count, ['ry', 'rz'], 'cx', 'linear', reps=proj_u_reps)
    num_u_params = u_ansatz.num_parameters
    print(f"  Unitary ansatz U(theta) has {num_u_params} parameters.")
    
    optimized_projectors = []
    for q_group in qubit_groups:
        q_group_str = "".join(map(str, q_group))
        print(f"\n  Optimizing projector for group {q_group}...")

        # Base projector construction (remains the same)
        s_identity_full = ['I'] * num_qubits
        s_z_on_group_full = ['I'] * num_qubits
        for q_idx in q_group: s_z_on_group_full[q_idx] = 'Z'
        pauli_list_for_proj = [("".join(s_identity_full)[::-1], 0.5), ("".join(s_z_on_group_full)[::-1], 0.5)]
        base_projector_spo = SparsePauliOp.from_list(pauli_list_for_proj)
        print(f"    Base projector for group {q_group} created.")

        # --- Gradient Function using Parameter-Shift Rule ---
        def calculate_gradient(params):
            gradient_vector = np.zeros_like(params)
            for i in range(len(params)):
                # Create parameter vectors for the +pi/2 and -pi/2 shifts
                params_plus = np.copy(params)
                params_plus[i] += np.pi / 2
                
                params_minus = np.copy(params)
                params_minus[i] -= np.pi / 2
                
                # Evaluate the cost function at the shifted points
                cost_plus = cost_func_for_grad(params_plus, u_ansatz, base_projector_spo, num_qubits, q_group, psi_initial_vector)
                cost_minus = cost_func_for_grad(params_minus, u_ansatz, base_projector_spo, num_qubits, q_group, psi_initial_vector)
                
                # Calculate the partial derivative
                gradient_vector[i] = 0.5 * (cost_plus - cost_minus)
            
            # print(f"  DEBUG: Gradient norm = {np.linalg.norm(gradient_vector)}") # Optional debug
            return gradient_vector

        # --- Optimization with a Gradient-Based Method ---
        init_params = np.random.rand(num_u_params) * 2 * np.pi
        print(f"    Starting L-BFGS-B optimization...")
        try:
            # Use a gradient-based optimizer like L-BFGS-B
            # It requires the cost function `fun` and the gradient function `jac`
            opt_res = minimize(
                fun=lambda p: cost_func_for_grad(p, u_ansatz, base_projector_spo, num_qubits, q_group, psi_initial_vector),
                x0=init_params, 
                method='L-BFGS-B',
                jac=calculate_gradient, # <-- Provide the gradient function
                tol=optimizer_tol, 
                options={'maxiter': optimizer_maxiter, 'disp': True}
            )
            
            print(f"    Optimization finished. Success: {opt_res.success}, Evals: {opt_res.nfev}, Final cost: {opt_res.fun:.6e}")
            
            if opt_res.success and opt_res.fun < 1e-5:
                u_circ_opt = u_ansatz.assign_parameters(opt_res.x)
                u_full_opt = QuantumCircuit(num_qubits)
                u_full_opt.compose(u_circ_opt, qubits=list(q_group), inplace=True)
                u_op_opt = Operator(u_full_opt)
                p_star_op = u_op_opt @ base_projector_spo @ u_op_opt.adjoint()
                optimized_projectors.append(p_star_op)
            else:
                print(f"    WARNING: Optimization for P_{q_group_str} did not meet success criteria. Message: {opt_res.message}")

        except Exception as e_opt:
            print(f"    ERROR: An exception occurred during optimization: {e_opt}")
            pass

    # ... (Rest of the function: Step 3 and 4 remain exactly the same) ...
    # ... (Find Ground State, Final Fidelity Calculation) ...
    print("\n[Step 3: Finding Ground State of Hamiltonian]")
    if not optimized_projectors:
        print("  No projectors were successfully optimized. Cannot continue.")
        return np.nan
    
    H_op = Operator(np.zeros((2**num_qubits, 2**num_qubits)))
    for proj in optimized_projectors:
        H_op += proj

    print("  Diagonalizing H matrix...")
    eigvals, eigvecs = np.linalg.eigh(H_op.data)
    gs_energy = eigvals[0]
    gs_gap = eigvals[1] - gs_energy if len(eigvals) > 1 else float('inf')
    gs_vector = Statevector(eigvecs[:, 0])
    
    print(f"  Ground state energy found: {gs_energy:.6e}")
    print(f"  Energy gap: {gs_gap:.6f}", "(Unique GS)" if gs_gap > 1e-9 else "(Degenerate GS)")

    print("\n[Step 4: Final Fidelity Calculation]")
    final_fidelity = state_fidelity(psi_initial_vector, gs_vector)
    print(f"  Fidelity(|psi_initial>, |found_GS>): {final_fidelity:.8f}")
    
    return final_fidelity

In [ ]:
fidelity = run_parent_hamiltonian_test(
    NUM_QUBITS, 
    psi_initial_vector, 
    PROJECTOR_QUBIT_COUNT,
    PROJECTOR_U_REPS, # Note: proj_rank is now calculated inside
    SEED,
    OPTIMIZER_TOL,
    OPTIMIZER_MAXITER)


--- Starting New Test Run (Seed: 42) ---
Config: N_qubits=8, Proj_Size=5, U_Reps=4

[Step 2: Defining and Optimizing Projectors with Parameter-Shift Rule]
  Unitary ansatz U(theta) has 50 parameters.

  Optimizing projector for group (0, 1, 2, 3, 4)...
    Base projector for group (0, 1, 2, 3, 4) created.
    Starting L-BFGS-B optimization...
    Optimization finished. Success: False, Evals: 85, Final cost: 2.054878e-02

  Optimizing projector for group (1, 2, 3, 4, 5)...
    Base projector for group (1, 2, 3, 4, 5) created.
    Starting L-BFGS-B optimization...


KeyboardInterrupt: 